In [3]:
import plotly.tools as tls
tls.embed('https://plot.ly/~chris/7365')

In [4]:
import pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display

import plotly.plotly as py # interactive graphing
from plotly.graph_objs import *
# from plotly.graph_objs import Bar, Scatter, Marker, Layout

In [10]:
display(pd.read_csv('311_Service_Requests_from_2010_to_Present.csv', nrows=2).head())
display(pd.read_csv('311_Service_Requests_from_2010_to_Present.csv', nrows=2).tail())

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,16068187,02/23/2010 12:00:00 AM,03/08/2010 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,PAINT - PLASTER,CEILING,RESIDENTIAL BUILDING,10304,101 WATER STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.627055,-74.077213,"(40.627055398012274, -74.07721261402538)"
1,16068188,02/23/2010 12:00:00 AM,03/11/2010 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,PAINT - PLASTER,CEILING,RESIDENTIAL BUILDING,10010,309 3 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.739133,-73.982917,"(40.73913261925362, -73.98291702868069)"


,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,16068187,02/23/2010 12:00:00 AM,03/08/2010 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,PAINT - PLASTER,CEILING,RESIDENTIAL BUILDING,10304,101 WATER STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.627055,-74.077213,"(40.627055398012274, -74.07721261402538)"
1,16068188,02/23/2010 12:00:00 AM,03/11/2010 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,PAINT - PLASTER,CEILING,RESIDENTIAL BUILDING,10010,309 3 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.739133,-73.982917,"(40.73913261925362, -73.98291702868069)"


In [13]:
!wc -l < 311_Service_Requests_from_2010_to_Present.csv # Number of lines in dataset

14644840


In [5]:
disk_engine = create_engine('sqlite:///311_8M.db') # Initializes database with filename

In [18]:
start = dt.datetime.now()
chunksize = 20000
j = 0
index_start = 1

for df in pd.read_csv('311_Service_Requests_from_2010_to_Present.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    df = df.rename(columns={c: c.replace(' ', '') for c  in df.columns}) # Remove spaces from columns
    
    df['CreatedDate'] = pd.to_datetime(df['CreatedDate']) # Convert to datetimes
    df['ClosedDate'] = pd.to_datetime(df['ClosedDate'])
    
    df.index += index_start
    
    # Removing uninteresting columns
    columns = ['Agency', 'CreatedDate', 'ClosedDate', 'ComplaintType', 'Descriptor', 'CreatedDate', 'ClosedDate', 'TimeToCompletion', 'City']
    
    for c in df.columns:
        if c not in columns:
            df = df.drop(c, axis=1)
    
    j+=1
    print('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))
          
    df.to_sql('data', disk_engine, if_exists='append')
    index_start = df.index[-1] + 1


/home/char/.virtualenvs/data-science/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning:

Columns (8,40,41,48) have mixed types. Specify dtype option on import or set low_memory=False.



8 seconds: completed 20000 rows


/home/char/.virtualenvs/data-science/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning:

Columns (8,17,39,40,41,43,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.



15 seconds: completed 40000 rows


/home/char/.virtualenvs/data-science/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning:

Columns (8,17,39,40,41,43,44,45,46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.



...

5533 seconds: completed 14500000 rows
5540 seconds: completed 14520000 rows
5548 seconds: completed 14540000 rows
5555 seconds: completed 14560000 rows
5563 seconds: completed 14580000 rows
5571 seconds: completed 14600000 rows


/home/char/.virtualenvs/data-science/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning:

Columns (8,17,39,40,41,47,48) have mixed types. Specify dtype option on import or set low_memory=False.



5579 seconds: completed 14620000 rows


/home/char/.virtualenvs/data-science/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning:

Columns (17,39,40,41,43,44,45,46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.



5587 seconds: completed 14640000 rows
5589 seconds: completed 14660000 rows


In [19]:
df = pd.read_sql_query('SELECT * FROM data LIMIT 3', disk_engine)
df.head()

,index,CreatedDate,ClosedDate,Agency,ComplaintType,Descriptor,City
0,1,2010-02-23 00:00:00.000000,2010-03-08 00:00:00.000000,HPD,PAINT - PLASTER,CEILING,STATEN ISLAND
1,2,2010-02-23 00:00:00.000000,2010-03-11 00:00:00.000000,HPD,PAINT - PLASTER,CEILING,NEW YORK
2,3,2010-02-23 00:00:00.000000,2010-03-15 00:00:00.000000,HPD,PAINT - PLASTER,CEILING,BROOKLYN


In [22]:
df = pd.read_sql_query('SELECT * FROM data LIMIT 3', disk_engine)
df.head()

,index,CreatedDate,ClosedDate,Agency,ComplaintType,Descriptor,City
0,1,2010-02-23 00:00:00.000000,2010-03-08 00:00:00.000000,HPD,PAINT - PLASTER,CEILING,STATEN ISLAND
1,2,2010-02-23 00:00:00.000000,2010-03-11 00:00:00.000000,HPD,PAINT - PLASTER,CEILING,NEW YORK
2,3,2010-02-23 00:00:00.000000,2010-03-15 00:00:00.000000,HPD,PAINT - PLASTER,CEILING,BROOKLYN


In [23]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'LIMIT 10 ', disk_engine)
df

,ComplaintType,Descriptor,Agency
0,PAINT - PLASTER,CEILING,HPD
1,PAINT - PLASTER,CEILING,HPD
2,PAINT - PLASTER,CEILING,HPD
3,PAINT - PLASTER,CEILING,HPD
4,PAINT - PLASTER,CEILING,HPD
5,Street Condition,Failed Street Repair,DOT
6,HEATING,HEAT,HPD
7,HEATING,HEAT,HPD
8,HEATING,HEAT,HPD
9,HEATING,HEAT,HPD


In [24]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'WHERE Agency = "NYPD" '
                       'LIMIT 10', disk_engine)
df.head()

,ComplaintType,Descriptor,Agency
0,Traffic/Illegal Parking,Commercial Overnight Parking,NYPD
1,Traffic/Illegal Parking,Commercial Overnight Parking,NYPD
2,Noise - Residential,Banging/Pounding,NYPD
3,Blocked Driveway,No Access,NYPD
4,Noise - Residential,Banging/Pounding,NYPD


In [10]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'WHERE Agency IN ("NYPD", "DOB") '
                       'LIMIT 10 ', disk_engine)
df.head()

,ComplaintType,Descriptor,Agency
0,General Construction/Plumbing,Fence - None/Inadequate,DOB
1,Building/Use,Illegal Conversion Of Residential Building/Space,DOB
2,General Construction/Plumbing,Debris - Falling Or In Danger Of Falling,DOB
3,Building/Use,Illegal Conversion Of Residential Building/Space,DOB
4,Plumbing,Failure To Retain Water/Improper Drainage- (LL...,DOB


In [30]:
df = pd.read_sql_query('SELECT DISTINCT City FROM data', disk_engine)
df.head()

,City
0,STATEN ISLAND
1,NEW YORK
2,BROOKLYN
3,ASTORIA
4,OZONE PARK


In [31]:
df = pd.read_sql_query('SELECT Agency, COUNT(*) as `num_complaints`'
                       'FROM data '
                       'GROUP BY Agency ', disk_engine)
df.head()

,Agency,num_complaints
0,3-1-1,48544
1,ACS,612
2,AJC,20
3,CAU,20
4,CCRB,26


In [7]:
df = pd.read_sql_query('SELECT Agency, COUNT(*) as `num_complaints`'
                       'FROM data '
                       'GROUP BY Agency '
                       'ORDER BY -num_complaints', disk_engine)

py.iplot([Bar(x=df.Agency, y=df.num_complaints)], filename='most common complaints by agency')

In [10]:
df = pd.read_sql_query('SELECT ComplaintType, COUNT(*) as `num_complaints`, Agency '
                       'FROM data '
                       'GROUP BY `ComplaintType` '
                       'ORDER BY -num_complaints', disk_engine)


most_common_complaints = df # used later
py.iplot({
    'data': [Bar(x=df['ComplaintType'], y=df.num_complaints)],
    'layout': { 
        'margin': {'b': 150}, # Make the bottom margin a bit bigger to handle the long text
        'xaxis': {'tickangle': 40}} # Angle the labels a bit
    }, filename='most common complaints by complaint type')

In [12]:
len(pd.read_sql_query('SELECT DISTINCT City FROM data', disk_engine))

2241

In [19]:
df = pd.read_sql_query('SELECT City, COUNT(*) as `num_complaints` '
                       'FROM data '
                       'GROUP BY `City` '
                       'ORDER BY -num_complaints '
                       'LIMIT 10 ', disk_engine)
df

,City,num_complaints
0,BROOKLYN,8628272
1,NEW YORK,5650168
2,BRONX,5277382
3,None,2078072
4,STATEN ISLAND,1382424
5,JAMAICA,425162
6,FLUSHING,335688
7,ASTORIA,277058
8,Jamaica,263686
9,RIDGEWOOD,204258


In [24]:
df = pd.read_sql_query('SELECT City, COUNT(*) as `num_complaints` '
                       'FROM data '
                       'GROUP BY `City` '
                       'COLLATE NOCASE '
                       'ORDER BY -num_complaints '
                       'LIMIT 11 ', disk_engine)
df

,City,num_complaints
0,BROOKLYN,8628272
1,NEW YORK,5650172
2,BRONX,5277382
3,None,2078072
4,STATEN ISLAND,1382424
5,JAMAICA,688848
6,FLUSHING,518704
7,ASTORIA,431654
8,RIDGEWOOD,312586
9,CORONA,225752


In [23]:
cities = list(df.City)
cities.remove(None)

In [26]:
traces = [] # the series in the graph - one trace for each city

for city in cities:
    df = pd.read_sql_query('SELECT ComplaintType, COUNT(*) as `num_complaints` '
                           'FROM data '
                           'WHERE City = "{}" COLLATE NOCASE '
                           'GROUP BY `ComplaintType` '
                           'ORDER BY -num_complaints'.format(city), disk_engine)

    traces.append(Bar(x=df['ComplaintType'], y=df.num_complaints, name=city.capitalize()))


In [28]:
py.iplot({'data': traces, 'layout': Layout(barmode='stack', xaxis={'tickangle': 40}, margin={'b': 150})}, filename='complaints by city stacked')

In [30]:
for trace in traces:
    trace['y'] = 100.*trace['y']/sum(trace['y'])

In [32]:
py.iplot({'data': traces, 
          'layout': Layout(
                barmode='group',
                xaxis={'tickangle': 40, 'autorange': False, 'range': [-0.5, 16]},
                yaxis={'title': 'Percent of Complaints by City'},
                margin={'b': 150},
                title='Relative Number of 311 Complaints by City')
         }, filename='relative complaints by city', validate=False)

In [33]:
df = pd.read_sql_query('SELECT ComplaintType, CreatedDate, City '
                       'FROM data '
                       'WHERE CreatedDate < "2014-11-16 23:47:00" '
                       'AND CreatedDate > "2014-11-16 23:45:00"', disk_engine)

df

,ComplaintType,CreatedDate,City
0,Derelict Vehicles,2014-11-16 23:46:00.000000,Jamaica
1,Noise - Commercial,2014-11-16 23:46:08.000000,ASTORIA
2,Noise - Street/Sidewalk,2014-11-16 23:45:10.000000,NEW YORK
3,Blocked Driveway,2014-11-16 23:45:43.000000,BROOKLYN
4,Derelict Vehicle,2014-11-16 23:46:57.000000,RIDGEWOOD
5,School Maintenance,2014-11-16 23:45:41.000000,BRONX
6,Benefit Card Replacement,2014-11-16 23:45:55.000000,None
7,Derelict Vehicles,2014-11-16 23:46:00.000000,Jamaica
8,Noise - Commercial,2014-11-16 23:46:08.000000,ASTORIA
9,Noise - Street/Sidewalk,2014-11-16 23:45:10.000000,NEW YORK


In [34]:
df = pd.read_sql_query('SELECT CreatedDate, '
                       'strftime(\'%H\', CreatedDate) as hour, '
                       'ComplaintType '
                       'FROM data '
                       'LIMIT 5 ', disk_engine)

df.head()

,CreatedDate,hour,ComplaintType
0,2010-02-23 00:00:00.000000,00,PAINT - PLASTER
1,2010-02-23 00:00:00.000000,00,PAINT - PLASTER
2,2010-02-23 00:00:00.000000,00,PAINT - PLASTER
3,2010-02-23 00:00:00.000000,00,PAINT - PLASTER
4,2010-02-23 00:00:00.000000,00,PAINT - PLASTER


In [35]:
df = pd.read_sql_query('SELECT CreatedDate, '
                               'strftime(\'%H\', CreatedDate) as hour,  '
                               'count(*) as `Complaints per Hour`'
                       'FROM data '
                       'GROUP BY hour', disk_engine)

df.head()

,CreatedDate,hour,Complaints per Hour
0,2010-02-23 00:00:00.000000,00,7728008
1,2010-02-24 01:38:41.000000,01,469830
2,2010-02-23 02:54:36.000000,02,324768
3,2010-02-23 03:13:00.000000,03,208944
4,2010-02-23 04:35:23.000000,04,172504


In [36]:
py.iplot({
    'data': [Bar(x=df['hour'], y=df['Complaints per Hour'])],
    'layout': Layout(xaxis={'title': 'Hour in Day'},
                     yaxis={'title': 'Number of Complaints'})}, filename='complaints per hour')

In [38]:
df = pd.read_sql_query('SELECT CreatedDate, '
                               'strftime(\'%H\', CreatedDate) as `hour`,  '
                               'count(*) as `Complaints per Hour`'
                       'FROM data '
                       'WHERE ComplaintType IN ("Noise", '
                                               '"Noise - Street/Sidewalk", '
                                               '"Noise - Commercial", '
                                               '"Noise - Vehicle", '
                                               '"Noise - Park", '
                                               '"Noise - House of Worship", '
                                               '"Noise - Helicopter", '
                                               '"Collection Truck Noise") '
                       'GROUP BY hour', disk_engine)

display(df.head(n=2))

py.iplot({
    'data': [Bar(x=df['hour'], y=df['Complaints per Hour'])],
    'layout': Layout(xaxis={'title': 'Hour in Day'},
                     yaxis={'title': 'Number of Complaints'},
                     title='Number of Noise Complaints in NYC by Hour in Day'
                    )}, filename='noise complaints per hour')

,CreatedDate,hour,Complaints per Hour
0,2010-02-24 00:47:10.000000,00,159146
1,2010-02-24 01:23:07.000000,01,125610


In [39]:
minutes = 15
seconds = 15*60

df = pd.read_sql_query('SELECT CreatedDate, '
                               'datetime(('
                                   'strftime(\'%s\', CreatedDate) / {seconds}) * {seconds}, \'unixepoch\') interval '
                       'FROM data '
                       'LIMIT 10 '.format(seconds=seconds), disk_engine)

display(df.head())

,CreatedDate,interval
0,2010-02-23 00:00:00.000000,2010-02-23 00:00:00
1,2010-02-23 00:00:00.000000,2010-02-23 00:00:00
2,2010-02-23 00:00:00.000000,2010-02-23 00:00:00
3,2010-02-23 00:00:00.000000,2010-02-23 00:00:00
4,2010-02-23 00:00:00.000000,2010-02-23 00:00:00


In [40]:
minutes = 15
seconds = minutes*60

df = pd.read_sql_query('SELECT datetime(('
                                   'strftime(\'%s\', CreatedDate) / {seconds}) * {seconds}, \'unixepoch\') interval ,'
                               'COUNT(*) as "Complaints / interval"'
                       'FROM data '
                       'GROUP BY interval '
                       'ORDER BY interval '
                       'LIMIT 500'.format(seconds=seconds), disk_engine)

display(df.head())
display(df.tail())

,interval,Complaints / interval
0,2010-01-01 00:00:00,2568
1,2010-01-01 00:15:00,30
2,2010-01-01 00:30:00,28
3,2010-01-01 00:45:00,50
4,2010-01-01 01:00:00,62


,interval,Complaints / interval
495,2010-01-06 04:15:00,4
496,2010-01-06 04:45:00,12
497,2010-01-06 05:00:00,10
498,2010-01-06 05:15:00,18
499,2010-01-06 05:30:00,18


In [41]:
py.iplot(
    {
        'data': [{
            'x': df.interval,
            'y': df['Complaints / interval'],
            'type': 'bar'
        }],
        'layout': {
            'title': 'Number of 311 Complaints per 15 Minutes'
        }
}, filename='complaints per 15 minutes')

In [42]:
hours = 24
minutes = hours*60
seconds = minutes*60

df = pd.read_sql_query('SELECT datetime(('
                                    'strftime(\'%s\', CreatedDate) / {seconds}) * {seconds}, \'unixepoch\') interval ,'
                                'COUNT(*) as "Complaints / interval"'
                       'FROM data '
                       'GROUP BY interval '
                       'ORDER BY interval '
                       'LIMIT 500 '.format(seconds=seconds), disk_engine)

display(df.head())
display(df.tail())

,interval,Complaints / interval
0,2010-01-01 00:00:00,5812
1,2010-01-02 00:00:00,7714
2,2010-01-03 00:00:00,11218
3,2010-01-04 00:00:00,18630
4,2010-01-05 00:00:00,16598


,interval,Complaints / interval
495,2011-05-11 00:00:00,11900
496,2011-05-12 00:00:00,11104
497,2011-05-13 00:00:00,10396
498,2011-05-14 00:00:00,5862
499,2011-05-15 00:00:00,4814


In [43]:
py.iplot(
    {
        'data': [{
            'x': df.interval,
            'y': df['Complaints / interval'],
            'type': 'bar'
        }],
        'layout': {
            'title': 'Number of 311 complaints per day'
        }
}, filename='complaints per day')